In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import codecs
import pickle
import pandas as pd
from datetime import datetime

In [3]:
restaurants = pd.read_excel('식당_분류_2024-08-29.xlsx')
restaurants.head()

,검색어,title,category,양식,브런치,일식,일식당,이탈리아음식,스파게티,파스타전문,...,한정식,초밥뷔페,두부요리,치킨,닭강정,순대,순댓국,복어요리,닭요리,프랑스음식
0,성수카페거리,37.5 시그니처 성수,양식>브런치,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,성수카페거리,탐광,음식점>일식>일식당,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,성수카페거리,타이거풀 성수,음식점>양식,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,성수카페거리,성수다락,"이탈리아음식>스파게티,파스타전문",0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,성수카페거리,bd버거 성수,음식점>햄버거,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
review = pd.read_excel('음식점_네이버_리뷰_24-08-28_1.xlsx')
review.head()

,keyword,review
0,37.5 시그니처 성수,"두 가지 다 너무 맛있어요!브런치는 잼, 휘핑, 시럽 곁들여 먹을 소스가 많아서 좋..."
1,37.5 시그니처 성수,"byeol9님, 맛있게 드셔주시고 칭찬과 함께 소중한 리뷰를 남겨주셔서 정말 감사드..."
2,37.5 시그니처 성수,부모님 모시고 간단히 먹을 곳 찾다가 들어왔는데 테이블끼리 간격도 넓고 음식도 빨리...
3,37.5 시그니처 성수,"누바48님, 맛있게 드셔주시고 칭찬과 함께 소중한 리뷰를 남겨주셔서 정말 감사드립니..."
4,37.5 시그니처 성수,아메라칸스타일 브런치가 땡겨서 찾아갔어요~ 찾고았던 딱 그 맛이였어요! 시간이 많이...


In [5]:
remove_review = ['남겨주셔서', '안녕하세요', '고객님', '소중한 리뷰', '방문해주셔서', '정성스러운 리뷰', '식사권을 제공', '솔직하게 남긴', '솔직하게 작성된', '온센텐동을', '두번째 방문인데', '두번째 방문이었는데', '최부자보쌈', '돼지 A세트']
remove = '|'.join(remove_review)

In [6]:
review = review[~review['review'].str.contains(remove, case=False, na=False)]
review.head()

,keyword,review
0,37.5 시그니처 성수,"두 가지 다 너무 맛있어요!브런치는 잼, 휘핑, 시럽 곁들여 먹을 소스가 많아서 좋..."
2,37.5 시그니처 성수,부모님 모시고 간단히 먹을 곳 찾다가 들어왔는데 테이블끼리 간격도 넓고 음식도 빨리...
4,37.5 시그니처 성수,아메라칸스타일 브런치가 땡겨서 찾아갔어요~ 찾고았던 딱 그 맛이였어요! 시간이 많이...
5,37.5 시그니처 성수,으앙너무 맛나고 몽블랑파스타 미쳒어요 신메뉴라고하는데 너무 꼬숩고 든든하게먹고갑니다...
7,37.5 시그니처 성수,예약하고 방문했어요~매장이 너무 예뻐요!! 너무 내스퇄✨️✨️몬테크리스토는 한정판매...


In [7]:
review.rename(columns={'keyword':'title'}, inplace=True)
review.head()

,title,review
0,37.5 시그니처 성수,"두 가지 다 너무 맛있어요!브런치는 잼, 휘핑, 시럽 곁들여 먹을 소스가 많아서 좋..."
2,37.5 시그니처 성수,부모님 모시고 간단히 먹을 곳 찾다가 들어왔는데 테이블끼리 간격도 넓고 음식도 빨리...
4,37.5 시그니처 성수,아메라칸스타일 브런치가 땡겨서 찾아갔어요~ 찾고았던 딱 그 맛이였어요! 시간이 많이...
5,37.5 시그니처 성수,으앙너무 맛나고 몽블랑파스타 미쳒어요 신메뉴라고하는데 너무 꼬숩고 든든하게먹고갑니다...
7,37.5 시그니처 성수,예약하고 방문했어요~매장이 너무 예뻐요!! 너무 내스퇄✨️✨️몬테크리스토는 한정판매...


In [8]:
review['review'] = review['review'].astype(str)

In [13]:
review_sum = review.groupby('title', as_index=False).agg({
    'review': lambda x: ', '.join(x)
    })

In [14]:
review_sum.head()

,title,review
0,101번지남산돈까스 남산본점,오랜만에 101번지 남산돈까스 남산본점에 왔는데 맛도 그대로 분위기도 그대로 너무 ...
1,37.5 시그니처 성수,"두 가지 다 너무 맛있어요!브런치는 잼, 휘핑, 시럽 곁들여 먹을 소스가 많아서 좋..."
2,8PIECE PIZZA,3명이서 크림 파스타와 루꼴라 시켜서 적당히 배부르게 먹고 왔어요 파스타는 양이 적...
3,bd버거 강남,"그릴드 쉬림프 와사비, 와사비쉬림프 버거 세트!! 무슨 일입니까?!! 너무 맛있고 ..."
4,bd버거 성수,풍자씨가 성수동에서 또간집 추천할 곳 알려 달라고 하면 보다버거 이야기할 거예요. ...


In [19]:
merged_review = pd.merge(restaurants, review_sum, on='title', how='left')
merged_review.head()

,검색어,title,category,양식,브런치,일식,일식당,이탈리아음식,스파게티,파스타전문,...,초밥뷔페,두부요리,치킨,닭강정,순대,순댓국,복어요리,닭요리,프랑스음식,review
0,성수카페거리,37.5 시그니처 성수,양식>브런치,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"두 가지 다 너무 맛있어요!브런치는 잼, 휘핑, 시럽 곁들여 먹을 소스가 많아서 좋..."
1,성수카페거리,탐광,음식점>일식>일식당,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,부추가 정말정말 맛있었던 탐광입니다자세한 이야기는 차차 하도록 하구요일단은 키오스크...
2,성수카페거리,타이거풀 성수,음식점>양식,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"친구 대학원 졸업 기념으로 방문했는데, 식당 분위기가 너무 좋고 음식이 맛있어서 좋..."
3,성수카페거리,성수다락,"이탈리아음식>스파게티,파스타전문",0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,NaN
4,성수카페거리,bd버거 성수,음식점>햄버거,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,풍자씨가 성수동에서 또간집 추천할 곳 알려 달라고 하면 보다버거 이야기할 거예요. ...


In [20]:
today_date = datetime.now().strftime('%Y-%m-%d')
filename = f'식당_리뷰추가{today_date}.xlsx'

In [21]:
merged_review.to_excel(filename, index=False)